In [1]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Button, Toggle, CheckboxGroup
from bokeh.plotting import figure, show

In [2]:
# Scatterplot: x, y from uniform(0,1)
# Add 1 particle on click

from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(
    title="Add 1 particle on click",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
    max_width=1000, 
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,1),
    y_range=(0, 1),
)

callback = CustomJS(args=dict(source=source), code="""
    source.data.x.push(Math.random())
    source.data.y.push(Math.random())

    // update the data source with local changes
    source.change.emit()
    """)

p.circle('x', 'y', source=source, size=10, fill_color="red")

# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

button.js_on_event('button_click', callback)

show(column(p, button))

In [3]:
# Scatterplot: x, y# Scatterplot: (x, y) from uniform(0,1) from uniform(0,1)
# Add 1 particle on click
# Send 100 particles at once
# Reset the screen

from bokeh.models import CustomJS, ColumnDataSource, Paragraph
from bokeh.plotting import figure, output_file, show

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(
    title="Add 1 particle on click or send 100 particles at once",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=500,
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,1),
    y_range=(0, 1),
)

p.circle('x', 'y', source=source, size=10, fill_color="red")

callback = CustomJS(args=dict(source=source), code="""
    source.data.x.push(Math.random())
    source.data.y.push(Math.random())

    // update the data source with local changes
    source.change.emit()
    """)

callback2 = CustomJS(args=dict(source=source), code="""
    
    let i = 0;
    for (let i = 0; i < 100; i++) {
    source.data.x.push(Math.random())
    source.data.y.push(Math.random()) 
    }
    
    // update the data source with local changes
    source.change.emit()
    """)

callback_reset = CustomJS(args=dict(source=source), code="""
    
    source.data.x = []
    source.data.y = []
    source.change.emit()
    """)

# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

button.js_on_event('button_click', callback)
button100.js_on_event('button_click', callback2)
resetbutton.js_on_event('button_click', callback_reset)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
show(column(p, button, button100, resetbutton))

In [4]:
# Scatterplot: x from uniform(0,1); y from normal(0,1)
# Add 1 particle on click
# Send 100 particles at once
# Reset the screen
# Continuous stream of particles

from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(
    title="Add 1 particle on click or send 100 particles at once",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=500,
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,1),
    y_range=(0, 1),
)

p.circle('x', 'y', source=source, size=10, fill_color="red")

#text = Paragraph(text="""
#    Some description
#    """, width=200, height=30)

# Def function of normal distribution
jscode_normalPDF = """
    function randn() {
      let u = 0, v = 0;
      
      while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
      while(v === 0) v = Math.random();
      
      let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2.0 * Math.PI * v );
      num = num / 10.0 + 0.5;           // Translate to 0 -> 1
      
      if (num > 1 || num < 0) return randn_bm() // resample between 0 and 1
      return num
    }
"""

callback = CustomJS(args=dict(source=source), code=jscode_normalPDF + """
    
    source.data.x.push(randn())
    source.data.y.push(Math.random())

    // update the data source with local changes
    source.change.emit()
    """)

callback2 = CustomJS(args=dict(source=source), code=jscode_normalPDF +"""
    
    let i = 0;
    for (let i = 0; i < 100; i++) {
    source.data.x.push(randn())
    source.data.y.push(Math.random()) 
    }
    
    // update the data source with local changes
    source.change.emit()
    """)

callback_reset = CustomJS(args=dict(source=source), code="""
    
    source.data.x = []
    source.data.y = []
    source.change.emit()
    """)

callback_stream = CustomJS(args=dict(source=source), code=jscode_normalPDF + """

    function sendparticle() {
        source.data.x.push(randn())
        source.data.y.push(Math.random())
        source.change.emit();
    }
    
    setInterval(sendparticle, 1000);
    """)

# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

# Continuous stream of particles
streamtoggle = Button(label="Send continuous stream", button_type="success")


button.js_on_event('button_click', callback)
button100.js_on_event('button_click', callback2)
resetbutton.js_on_event('button_click', callback_reset)
streamtoggle.js_on_event('button_click', callback_stream)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
show(column(p, button, button100, resetbutton, streamtoggle))

In [8]:
# Scatterplot: x from uniform(0,1); y from normal(0,1)
# Add 1 particle on click
# Send 100 particles at once
# Reset the screen
# Continuous stream of particles + jde vypnout

from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_file, show

source = ColumnDataSource(data=dict(x=[], y=[]))

p = figure(
    title="Add 1 particle on click or send 100 particles at once",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=500,
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,1),
    y_range=(0, 1),
)

p.circle('x', 'y', source=source, size=10, fill_color="red")

# Buttons ----------------------------------------------------------------------
# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

# Continuous stream of particles
streamtoggle = Button(label="Send continuous stream", button_type="success")
# -----------------------------------------------------------------------------
# JS CALLBACKS

# Def function of normal distribution
jscode_normalPDF = """
    function randn() {
      let u = 0, v = 0;
      
      while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
      while(v === 0) v = Math.random();
      
      let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2.0 * Math.PI * v );
      num = num / 10.0 + 0.5;           // Translate to 0 -> 1
      
      if (num > 1 || num < 0) return randn_bm() // resample between 0 and 1
      return num
    }
"""

callback = CustomJS(args=dict(source=source), code=jscode_normalPDF + """
    
    source.data.x.push(randn())
    source.data.y.push(Math.random())

    // update the data source with local changes
    source.change.emit()
    """)

callback2 = CustomJS(args=dict(source=source), code=jscode_normalPDF +"""
    
    let i = 0;
    for (let i = 0; i < 100; i++) {
    source.data.x.push(randn())
    source.data.y.push(Math.random()) 
    }
    
    // update the data source with local changes
    source.change.emit()
    """)

callback_reset = CustomJS(args=dict(source=source), code="""
    
    source.data.x = []
    source.data.y = []
    source.change.emit()
    """)

callback_stream = CustomJS(args=dict(source=source, streamtoggle=streamtoggle), code="""

    function sendparticle() {
        source.data.x.push(Math.random());
        source.data.y.push(Math.random());
        source.change.emit();
    }
    
    if (streamtoggle.button_type == "success") {
        const interval = setInterval(sendparticle, 300);
        sendparticle();
        source.interval = interval;
        streamtoggle.button_type = "danger";
        streamtoggle.label = "Stop stream";
        
    } else {
        clearInterval(source.interval);
        streamtoggle.button_type = "success";
        streamtoggle.label = "Send continuous stream";
    }
""")
# -------------------------------------------------------------------------------
# Callbacks to objects

button.js_on_event('button_click', callback)
button100.js_on_event('button_click', callback2)
resetbutton.js_on_event('button_click', callback_reset)
streamtoggle.js_on_event('button_click', callback_stream)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
show(column(p, button, button100, resetbutton, streamtoggle))

In [6]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(width=400, height=400, x_range=(0, 1), y_range=(0, 1))

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const f = cb_obj.value
    const x = source.data.x
    const y = Array.from(x, (x) => Math.pow(x, f))
    source.data = { x, y }
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

In [49]:
# Plot of a normal distribution function
# https://justinbois.github.io/bootcamp/2021/lessons/l30_javascript_for_bokeh.html

# Initial setup of the sliders is the same
mu_slider = bokeh.models.Slider(title="mu", start=-5, end=5, step=0.1, value=0)
sigma_slider = bokeh.models.Slider(title="rho", start=0.1, end=5, step=0.1, value=1)

# Setup of x and y values is also the same
x = np.linspace(-10, 10, 200)
pdf = scipy.stats.norm.pdf(x, loc=0, scale=1)

# Set up the plot as a Bokeh figure
p = bokeh.plotting.figure(
    frame_width=300,
    frame_height=200,
    x_axis_label="x",
    y_axis_label="f(x; μ, σ)",
    x_range=[-10, 10],
)




for (var i = 0; i < lastValueOfiYouWant; i++) {
    // body of for loop
}

# JavaScript code for the callback stored as a string
jscode = """
function normalPDF(x, mu, sigma) {
    var expArg = (x - mu) ** 2 / 2.0 / sigma ** 2;
    return Math.exp(-expArg) / sigma / Math.sqrt(2 * Math.PI);
}

// For convenience, get views into the ColumnDataSource data
var x = cds.data['x'];
var pdf = cds.data['pdf'];

// Pull the values off of the sliders
var mu = mu_slider.value;
var sigma = sigma_slider.value;

// Loop through and update
var xLen = x.length;
for (var i = 0; i < xLen; i++) {
    pdf[i] = normalPDF(x[i], mu, sigma);
}

// Once the ColumnDataSource is changed, we need to use its
// .change.emit() method to update the plot
cds.change.emit();
"""

args = dict(cds=cds, mu_slider=mu_slider, sigma_slider=sigma_slider)

# Connect the callback to the sliders
mu_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
sigma_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

slider_created_using_panel.jscallback(value=jscode, args=args)

bokeh.io.show(layout)

SyntaxError: invalid syntax (1700258876.py, line 24)

In [11]:
# Tabs in a panel // to be used later"

from bokeh.io import show
from bokeh.models import Panel, Tabs
from bokeh.plotting import figure

p1 = figure(width=300, height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(width=300, height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title="line")

show(Tabs(tabs=[tab1, tab2]))

In [75]:
# Change text on a button click, display text every second and stop displaying
from bokeh.models import CustomJS
from bokeh.layouts import row, column
from bokeh.models.widgets import Div, Button

op="FirstText"

Output= Div(text="""<font size="4">""" + op + """</font>""", width=200, height=20)

ResponseButton = Button(label="Change text")
ResponseButton2 = Button(label="Add text")
STOPButton = Button(label="Stop text")

ChangeText = """
    o1.text="<font size='4'>"+"NewText"+"</font>";
"""

AddText = """
    o1.text+="<font size='4'>" + "NewText"+"</font>";
"""

StartText = """
    function displayText() {
        o1.text+="Hello";
    }
    
    setInterval(displayText, 1000);
"""

StopText = """
    const animate = setInterval(displayText, 1000);

    function displayText() {
        o1.text+="Hello";
    }

    function StopFunction() {
        clearInterval(animate);
    }
"""

StartStopText = """
    function setColor() {
      let x = document.body;
      x.style.backgroundColor = x.style.backgroundColor == "yellow" ? "pink" : "yellow";
    }
    
    var animate = setInterval(setColor, 1000);
    
    function stop(){
        clearInterval(animate);
    } 
"""


callback = CustomJS(args={'o1':Output}, code=ChangeText)
callback2 = CustomJS(args={'o1':Output}, code=AddText)
callback3 = CustomJS(args={'o1':Output}, code=StartText)
stopcallback = CustomJS(args={'o1':Output}, code=StartStopText)

ResponseButton.js_on_event('button_click', callback)
#ResponseButton2.js_on_event('button_click', callback2)
ResponseButton2.js_on_event('button_click', callback3)
#STOPButton.js_on_event('button_click', stopcallback)

layout=(column(ResponseButton, ResponseButton2, STOPButton, Output))

show(layout)

In [76]:
# Change text on a button click, display text every second and stop displaying // not working
from bokeh.models import CustomJS
from bokeh.layouts import row, column
from bokeh.models.widgets import Div, Button

op="FirstText"
Output= Div(text="""<font size="4">""" + op + """</font>""", width=200, height=20)

toggle = Toggle(label="Play/Stop Animation", button_type="success", active=False)
jscode = """
        var f = cb_obj.active;
        
        if(f == true){
            animate = setInterval(displayText, 500);             
        } else {
            clearInterval(animate);
        }
        
        function displayText() {
        o1.text+="Hello";
        }
"""

callback = CustomJS(args={'o1':Output}, code=jscode)
toggle.js_on_event('button_click', callback)

layout=(column(toggle, Output))

show(layout)

In [58]:
# Bokeh rainsdrop animation
# https://observablehq.com/@stas-sl/bokeh-raindrops-animation